In [15]:
import pandas as pd
import numpy as np

data = pd.read_csv("./data/features3_train_2.csv")
X = data[data.columns[:-1]]
y = data['VV']
# sklearn impute advanced(IterativeImpute)

In [16]:
random_seed = 42
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=random_seed)

In [17]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2
from sklearn.metrics import f1_score
vt = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_train_vt = vt.fit_transform(X_train)
X_valid_vt = vt.transform(X_valid)
X_train_vt

array([[ 0.000000e+00,  0.000000e+00,  1.701805e+06, ...,  3.300000e+01,
         1.360000e+02,  1.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ..., -1.000000e+00,
         1.400000e+01,  1.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  0.000000e+00],
       ...,
       [ 0.000000e+00,  1.000000e+00,  0.000000e+00, ...,  0.000000e+00,
         0.000000e+00,  1.000000e+00],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00, ..., -1.000000e+00,
        -1.000000e+00,  1.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  2.600000e+05, ..., -1.000000e+00,
         0.000000e+00,  0.000000e+00]])

In [18]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

feature_selection_tree = ExtraTreesClassifier(n_estimators=50)
feature_selection_tree.fit(X_train_vt, y_train)
feature_selection_tree.feature_importances_
feature_selection_model = SelectFromModel(feature_selection_tree, prefit=True)
X_train_selected = feature_selection_model.transform(X_train_vt)
X_valid_selected = feature_selection_model.transform(X_valid_vt)
X_train_selected


array([[ 1.7160000e+03,  1.2432000e+05,  4.2000000e+01, ...,
         2.0782000e+04,  1.7022000e+04,  6.8305800e+05],
       [ 1.0800000e+02,  1.8530500e+05,  2.1294632e+01, ...,
         5.4000000e+04, -1.0000000e+00,  2.1580500e+05],
       [ 6.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 1.9000000e+01,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00, -1.0000000e+00,  0.0000000e+00],
       [-1.0000000e+00,  1.2560000e+03,  1.0000000e+00, ...,
         1.7058000e+04,  0.0000000e+00, -1.0000000e+00],
       [ 7.0620000e+03,  0.0000000e+00,  0.0000000e+00, ...,
         3.0851700e+04,  0.0000000e+00,  0.0000000e+00]])

In [19]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE,SVMSMOTE
oversample = SVMSMOTE()
X_over, y_over = oversample.fit_resample(X_train_selected, y_train)
y_over.value_counts()

0    19170
1    19170
Name: VV, dtype: int64

In [20]:
from sklearn.ensemble import RandomForestClassifier
random_seed = 42
rfc = RandomForestClassifier(random_state=random_seed)
rfc.fit(X_over, y_over)

y_train_pred = rfc.predict(X_over)
y_valid_pred = rfc.predict(X_valid_selected)

from sklearn.metrics import f1_score
print(f1_score(y_over, y_train_pred))
print(f1_score(y_valid, y_valid_pred))

0.9976535613724059
0.22822822822822822


In [26]:
from lightgbm.sklearn import LGBMClassifier
clf = LGBMClassifier(boosting_type='dart')
clf.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [27]:
from sklearn.experimental.enable_halving_search_cv import HalvingGridSearchCV
param_id = {
        "n_estimators": [10000],
        "learning_rate":  np.linspace(start=0.01, stop=0.3, num=10),
        "num_leaves": np.linspace(start=20, stop=3000, num=50),
        "max_depth": np.linspace(start=3, stop=12, num=10),
        "min_data_in_leaf": [int(x) for x in np.linspace(start=200, stop=10000, num=100)],
        "max_bin": np.linspace(start=200, stop=300, num=10),
        "lambda_l1": [0.01, 0.001, 0.1, 1, 10, 100],
        "lambda_l2": [0.01, 0.001, 0.1, 1, 10, 100],
        "min_gain_to_split": [0.1, 0.2, 0.3, 0.4, 0.5, None],
        "bagging_fraction": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
        "bagging_freq": [0, 1],
        "feature_fraction": []
}
grid_search = HalvingGridSearchCV(estimator=clf, param_grid=param_id, cv=5, scoring='f1', random_state=42)

grid_search.fit(X_over, y_over)

[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] Unknown parameter: max_leaf_nodes
[LightGBM] [Warning] min_data_in_leaf is set=100, 

HalvingGridSearchCV(estimator=LGBMClassifier(),
                    param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                                'max_leaf_nodes': [20, 30, 50, 100, 200, 300,
                                                   500, 800, 1000, 2000, 3000],
                                'min_data_in_leaf': [100, 200, 300, 500, 1000],
                                'n_estimators': [1, 20, 50, 100, 200]},
                    random_state=42,
                    refit=<function _refit_callable at 0x7f8a3699d040>,
                    scoring='f1')

In [25]:
grid_search.best_score_
grid_search.best_params_
y_valid_pred = grid_search.predict(X_valid_selected)
print(f1_score(y_valid, y_valid_pred))

0.2578616352201258


In [ ]:
#max_depth
#min_sample_split
#max_leaf_nodes
#min_samples_leaf
#n_estimators
#max_sample (bootstrap sample)
#max_features

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import validation_curve
param_range = [int(x) for x in np.linspace(start=3, stop=15, num=13)]
train_scores, test_scores = validation_curve(estimator=rfc,X=X_train_selected, y=y_train,
                                         param_name='max_depth',
                                         param_range=param_range,
                                         scoring='f1',
                                             cv=10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
plt.plot(param_range, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(param_range, test_mean, color='green', marker='s', linestyle='--', markersize=5, label='Validation Accuracy')
plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.grid()
plt.xlabel("Max Depth")
plt.ylabel("F1 Score")
plt.legend(loc='upper left')
plt.ylim([0.1, 1.0])
plt.show()